In [112]:
from py2neo import Graph, Node, Relationship
import numpy as np
import pandas as pd

In [141]:
grapp = Graph()

In [ ]:
# query = '''
#     LOAD CSV WITH HEADERS FROM '../doh/mimic3/data/ADMISSIONS.csv' AS line
#     WITH line
#     WHERE line.House IS NOT NULL
#     MERGE (person:Person{Name:line.Name})
#     MERGE (house:House{Name:line.House})
#     MERGE (person) -[:belongs_to]-> (house)
# '''

# ”’
# graph.run(query)

In [114]:
patients = pd.read_csv('../doh/mimic3/data/PATIENTS.csv')
admissions = pd.read_csv('../doh/mimic3/data/ADMISSIONS.csv')

In [115]:
unique_pats = admissions.SUBJECT_ID.unique()
profs = pd.merge(admissions,patients,how='left',on='SUBJECT_ID').\
            drop(['ROW_ID_y','ROW_ID_x','DOB','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'], axis=1)

In [142]:
profs.shape,  admissions.shape,unique_pats.shape

((58976, 19), (58976, 19), (46520,))

In [117]:
grouped = profs.groupby(['SUBJECT_ID'])

In [144]:
tx = graph.begin()
# for sub_id in unique_pats[0:1000]:
for sub_id in unique_pats:
    pat_acts = grouped.get_group(sub_id).sort_values(by=['ADMITTIME'])
    last_act = pat_acts.tail(1)
    p = Node("Patient", pid=last_act.SUBJECT_ID.values[0],insurance=last_act.INSURANCE.values[0],\
             religion=last_act.RELIGION.values[0],marital=last_act.MARITAL_STATUS.values[0],
             ethnicity=last_act.ETHNICITY.values[0],gender=last_act.GENDER.values[0])
    tx.create(p)
    
    for index, row in pat_acts.iterrows():
        act = Node("Activity", hadm=row['HADM_ID'],admit_time=row['ADMITTIME'], disch_time=row['DISCHTIME'],\
        admit_type=row['ADMISSION_TYPE'],admit_loc=row['ADMISSION_LOCATION'],disch_loc=row['DISCHARGE_LOCATION'],
                  diag=row['DIAGNOSIS'])
        tx.create(act)
        hsp = Relationship(p, "Hospital", act)
        tx.create(hsp)
        p = act
tx.commit()   


In [140]:
graph.data("MATCH (n:Patient)-[:Hospital]->(act1)-[:Hospital]->(act) RETURN n,act1.admit_type,act.admit_type LIMIT 25")

[{u'act.admit_type': u'EMERGENCY',
  u'act1.admit_type': u'ELECTIVE',
  u'n': (a44e596:Patient {ethnicity:"WHITE",gender:"M",insurance:"Medicare",marital:"MARRIED",pid:23,religion:"CATHOLIC"})},
 {u'act.admit_type': u'EMERGENCY',
  u'act1.admit_type': u'EMERGENCY',
  u'n': (d39da98:Patient {ethnicity:"WHITE",gender:"M",insurance:"Medicare",marital:"MARRIED",pid:34,religion:"CATHOLIC"})},
 {u'act.admit_type': u'EMERGENCY',
  u'act1.admit_type': u'EMERGENCY',
  u'n': (e2e6b89:Patient {ethnicity:"WHITE",gender:"M",insurance:"Medicare",marital:"MARRIED",pid:36,religion:"NOT SPECIFIED"})},
 {u'act.admit_type': u'EMERGENCY',
  u'act1.admit_type': u'EMERGENCY',
  u'n': (f6bbaec:Patient {ethnicity:"WHITE",gender:"M",insurance:"Private",marital:"MARRIED",pid:357,religion:"NOT SPECIFIED"})},
 {u'act.admit_type': u'EMERGENCY',
  u'act1.admit_type': u'EMERGENCY',
  u'n': (c6458c2:Patient {ethnicity:"MULTI RACE ETHNICITY",gender:"M",insurance:"Medicare",marital:"MARRIED",pid:361,religion:"CATHOLIC"

In [178]:

# for s in patients.dtypes:
#     print(s.type) 
#     print(s.type== np.object_) 
if any(s.type== np.object_ for s in patients.dtypes):
     print("Object") 

Object


In [181]:
 patients.select_dtypes(exclude=['object']).iloc[0].tolist()

[234, 249, 0]